In [6]:
from langchain_ollama.llms import OllamaLLM
from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma

ollama = OllamaLLM(base_url="http://localhost:11434", model="phi3")
oembed = OllamaEmbeddings(base_url="http://localhost:11434", model="nomic-embed-text")

vectorstore = Chroma(
    collection_name="med_notes",
    embedding_function=oembed,
    persist_directory="./med_db",  # Where to save data locally, remove if not necessary
)

### Define Prompt Template


In [10]:
from langchain.prompts import ChatPromptTemplate

template = """
    <System>
    You are a Medical Advisor conversational AI. 
    YOU ONLY ANSWER QUESTIONS ABOUT medicals. 
    If question is not related to medical you do not know the answer to a question, you truthfully say that you do not know.
    You have access to information provided by the human in the "document" tags below to answer the question, and nothing else.
    </System>
        
    <documents>
    {context}
    </documents>

    <instruction>
    Your answer should ONLY be drawn from the provided search results above, never include answers outside of the search results provided.
    Once you are done extracting relevant quotes, answer the question. 
    </instruction>

    <question>
    {question}
    </question>
    
    """

prompt = ChatPromptTemplate.from_template(template)

In [19]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(ollama, 
                                       retriever=vectorstore.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": prompt}
                                       )

res = qa_chain.invoke({"query": "What causing the head to have clamping sensation?"})
print(res['result'])

It seems that there is no specific information provided in the documents about a 'clamping' sensation being caused by migraines with brainstem aura or basilar artery migraines. The symptoms listed for these types of headaches, according to the "Question" and corresponding answer from one document include: Nausea Vomiting Sensitivity to light and sound Cold hands or feet Dizziness Double vision or graying of vision Slurred speech or trouble speaking Temporary blindness Loss of balance Confusion Trouble hearing Body tingling Loss of consciousness Aura symptoms may last between 5 minutes and 1 hour. While dizziness, double vision (which could potentially be interpreted as a 'clamping' sensation in the context), cold hands or feet are mentioned, there is no direct mention that these specific symptoms cause head clamping. Therefore, I don’t know this information based on provided documents


In [18]:
res

{'query': 'What cause headache?',
 'result': 'Based on your query and available information from WebMD:\n\n- Migraine with brainstem aura or MBA (formerly known as basilar migraines) are characterized by symptoms like dizziness, double vision, lack of coordination caused by changes called an "aura," which can happen about 10 minutes to 45 minutes before the headache begins. The pain typically starts on one side of the head and gradually spreads and intensifies.\n- Common triggers include alcohol, stress, lack of sleep, certain food items (e.g., dairy products), chocolate, eggs, wheat, rye, tomatoes, oranges, as well as egg consumption in some people with migraines who have an associated condition called \'migralepsy.\'\n- If the individual doesn\'t respond to other treatments and experiences 4 or more days of headache per month, preventive medication might be recommended.',
 'source_documents': [Document(metadata={}, page_content='"Question":"What causes a migraine with brainstem aura?